In [ ]:
import pandas as pd
import numpy as np
import time
import concurrent.futures

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
drive.CreateFile({'id':'1YAAvPUaPeBIddkuVzWgw7fhPPcT2uJTY'}).GetContentFile('billboard_dataset_unique.csv')
df_billboard = pd.read_csv("billboard_dataset_unique.csv").drop('Unnamed: 0',axis=1)

drive.CreateFile({'id':'1kEYc4W4-rPhWtFJYpkDAWaAOGGO-eB3B'}).GetContentFile('dataset2.csv')
df_songs = pd.read_csv("dataset2.csv")

In [ ]:
df_songs_bak = df_songs.copy()

In [ ]:
# elimino tracce precedenti al 1960
mask = df_songs.year > 1959
df_songs = df_songs[mask]

# converto colonna 'release_date' in datetime
df_songs.release_date = pd.to_datetime(df_songs.release_date,format="%Y-%m-%d",exact=False)

# ordino il df per data
df_songs = df_songs.sort_values('release_date',ascending=False)

# elimino eventuali id ripetuti
df_songs = df_songs.drop_duplicates(subset='id',keep='last')    # keep='last' considerando che ho ordinato il df in senso discendente 

# elimino ripubblicazioni (tenendo solo traccia più vecchia)
df_songs = df_songs.drop_duplicates(subset=['artists','name'],keep='last')    # keep='last' considerando che ho ordinato il df in senso discendente 

# converto colonna artists da stringa a lista di stringhe
df_songs.artists = df_songs.artists.apply(eval)

In [ ]:
# inserisco i due target 'hit' e 'weeks' nel dataframe delle canzoni 

# 1) creo 2 nuove colonne nel dataframe delle canzoni (df_songs)

hit_array = np.zeros(df_songs.id.count())
weeks_array = np.zeros(df_songs.id.count())

df_songs.insert(19,'hit',hit_array)
df_songs.insert(20,'weeks',weeks_array)

In [ ]:
def hit_match(i,df_billboard,df_songs):
  entry_title = df_billboard.title[i]
  entry_weeks = df_billboard.weeks[i]
  entry_artist = df_billboard.artist[i]

  output = {}

  print("Checking %s by %s ..." % (entry_title,entry_artist))

  mask = df_songs.name.str.lower() == entry_title.lower()
  sub_df = df_songs[mask]    # creo sotto-dataframe (di df_songs) con le canzoni con lo stesso titolo della hit in esame

  for k in range(sub_df.shape[0]):   # per ogni traccia del sotto-dataframe (sub_df)
    artist_list = sub_df.artists.iloc[k]
    id = sub_df.id.iloc[k]

    for j in range(len(artist_list)):
      if(artist_list[j] in entry_artist):
        match = True
      else:
        match = False
        break
    if(match == True):
      print("*** %s by %s it's a HIT! ***" % (entry_title,entry_artist))
      output.update({id:entry_weeks})

  return output

In [ ]:
time_0 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
  results = [executor.submit(hit_match, i,df_billboard,df_songs) for i in range(df_billboard.shape[0])]

  hit_dict = {}

  for f in concurrent.futures.as_completed(results):
    hit_dict.update(f.result())

  print("Completato in %.4f secondi" % (time.perf_counter()-time_0))

Output streaming troncato alle ultime 5000 righe.
*** Tijuana Taxi by Herb Alpert & The Tijuana Brass it's a HIT! ***
Checking You Didn't Have To Be So Nice by The Lovin' Spoonful ...
*** You Didn't Have To Be So Nice by The Lovin' Spoonful it's a HIT! ***
Checking Sandy by Ronny And The Daytonas ...
Checking It Was A Very Good Year by Frank Sinatra ...
*** It Was A Very Good Year by Frank Sinatra it's a HIT! ***
Checking Attack by The Toys ...
Checking Tell Me Why by Elvis Presley With The Jordanaires ...
Checking Are You There (With Another Girl) by Dionne Warwick ...
Checking Second Hand Rose by Barbra Streisand ...
Checking Michael by The C.O.D.'s ...
*** Second Hand Rose by Barbra Streisand it's a HIT! ***
Checking I'm Too Far Gone (To Turn Around) by Bobby Bland ...
Checking Recovery by Fontella Bass ...
Checking My Generation by The Who ...
*** My Generation by The Who it's a HIT! ***
Checking Michelle by Billy Vaughn And His Orchestra ...
Checking Since I Lost The One I Love by

In [ ]:
df_songs_bak = df_songs.copy()

In [ ]:
for key in list(hit_dict):
    mask = df_songs.id == key

    df_songs[mask] = df_songs[mask].apply(lambda x: 1 if x.name == 'hit' else x)
    df_songs[mask] = df_songs[mask].apply(lambda x: hit_dict[key] if x.name == 'weeks' else x)

In [ ]:
from google.colab import  drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
df_songs.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/dataset2_X_billboard.csv')

In [ ]:
df_songs.id.count()

111546